In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [5]:
import pandas as pd
import deepmatcher as dm
from utils.explainer import Explainer
from utils.patternDiscovery import mineRules

In [7]:
pos_nn = pd.read_csv('experiment_results/testposNN_RNN.csv')
train_neg = pd.read_csv('../../Structured/Amazon-Google/train_negatives.csv')
association_rules = mineRules(pos_nn,train_neg,'title',min_confidence=0.8,min_support=0.1)
association_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_isleft,consequents_isright
5,(L_microsoft),(R_microsoft),0.289062,0.289062,0.289062,1.00,3.459459,0.205505,inf,True,True
77,"(L_windows, L_microsoft)",(R_microsoft),0.171875,0.289062,0.171875,1.00,3.459459,0.122192,inf,True,True
204,(L_office),(R_microsoft),0.156250,0.289062,0.148438,0.95,3.286486,0.103271,14.21875,True,True
206,"(L_office, L_microsoft)",(R_microsoft),0.148438,0.289062,0.148438,1.00,3.459459,0.105530,inf,True,True
213,"(L_office, L_windows)",(R_microsoft),0.101562,0.289062,0.101562,1.00,3.459459,0.072205,inf,True,True


In [9]:
association_rules_onprice = mineRules(pos_nn,train_neg,'price',min_confidence=0.8,min_support=0.05)
association_rules_onprice.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_isleft,consequents_isright
1,(L_239.99),(R_109.99),0.080537,0.087248,0.067114,0.833333,9.551282,0.060087,5.47651,True,True


## Test patterns

In [10]:
rnn_model = dm.MatchingModel(attr_summarizer='rnn')
rnn_model.load_state('../../models/amazongoogle_rnn.pth')
explainer = Explainer(rnn_model,5)

In [ ]:
p1 =['microsoft windows','microsoft windows']
explainer.testRobustness(testset_path='../../Structured/Amazon-Google/test_positives.csv',
                         true_label=1,attribute='title',substitute_values=p1,
                         strategy='concat')

In [12]:
import numpy as np
p2 = ['239.99','109.99']
explainer.testRobustness(testset_path='../../Structured/Amazon-Google/positives.csv',
                         true_label=1,attribute='price',substitute_values=p2,
                         strategy='replace')

===>  PREDICT Epoch 18
Finished Epoch 18 || Run Time:    0.6 | Load Time:    1.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00

===>  PREDICT Epoch 18
Finished Epoch 18 || Run Time:    0.6 | Load Time:    1.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(990, 1009)

## Inspect NN on critical attributes for negatives

In [ ]:
negNN = pd.read_csv('experiment_results/testnegNN_RNN.csv')
trainPos = pd.read_csv('../../Structured/Amazon-Google/train_positives.csv')
transactions = preProcessNeighbors(negNN,trainPos,'price')

In [ ]:
itemsets, rules = apriori(transactions, min_support=0.05,  min_confidence=0.8)
rules_rhs = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs)==1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule) # Prints the rule and its confidence, support, lift, ...

In [ ]:
p1 = ['35.98','39.99']
explainer.testRobustness(testset_path='../../Structured/Amazon-Google/test_negatives.csv',
                         true_label=0,attribute='price',substitute_values=p1,
                         strategy='replace')

In [ ]:
p2 = ['22.99','19.99']
explainer.testRobustness(testset_path='../../Structured/Amazon-Google/test_negatives.csv',
                         true_label=0,attribute='price',substitute_values=p2,
                         strategy='replace')